In [4]:
import pandas as pd 
from extractFeatures import getTexts

csv_data = pd.read_csv('/Users/amycweng/Digital Humanities/Early-Modern-London/Sermons_Info/sermons.csv')
tcpIDs = [ _ for _ in csv_data['id']]

Extract the entire body text from an XML document.  

In [ ]:
from getTexts import *
# EP: /Users/amycweng/Digital Humanities/eebotcp
# TCP: /Users/amycweng/Digital Humanities/TCP
outputfolder = 'charityTXT'
convert(tcpIDs,outputfolder)
print(underscores)
convert(underscores,outputfolder)

'''
TCP charity texts that are not in EP: 
'B33867', 'B13871', 'B13868', 'B13878', 'B13862', 'B13860', 
'B09696', 'A86972', 'A19579', 'A09957'
'''

Reading the sermons among the charity texts from their text files into a dictionary. Stopwords are removed in the process.

In [8]:
sermons = getTexts('/Users/amycweng/Digital Humanities/charityTXT',tcpIDs,True)
print(len(sermons))

70


Extracting the context windows of 20 words around key terms into an output file. 

In [10]:
groups = {0: 'charity|charitable|charitie', 
        1: 'beneficience|bequeath|bestow|liberality|benevolence|goodwill| alm | almes |kindness|almsdeed',
        2: 'poor|poverty|rich|wealthy|wealth|beggar|orphan|needy|labourer|impotent|bankrup|penury|meanness|prosperity|bounty|impouerish|impoverish',
        3: 'rate|usury|usurer|money|commodity|talon|talent|coin|bullion|exchange|shilling|farthing| pound |penny|purchase',
        4: 'credit|creditor|loan|lend',
        5: ' debt|debtor|borrow|pledge|pawn',
        6: 'industry|industrious|thrift|labor|labour|occupation|profit|profitable|commodious',
        7: 'prison|imprisonment',
        8: 'hospital|almshouse|orphanage',
        9: 'city|citizen|london',
        10: 'vagrant|vagrancy'
        }
search = [part for part in groups.values()]
search = '|'.join(search)

In [14]:
outdir = '/Users/amycweng/Digital Humanities/charityContexts'
for tcpID,text in sermons.items(): 
    contexts = []
    text = text.split(' ')
    indices = []
    for idx, word in enumerate(text):
        toContinue = False
        if re.match(search,word):  
            for i in indices:
                if idx in range(i-10,i+10):
                    toContinue = True
            if not toContinue: 
                indices.append(idx) 
                contexts.append(' '.join(text[idx-10:idx+10]))
    with open(f'{outdir}/{tcpID}.txt','w+') as file: 
        file.write(' '.join(contexts))

Searching for sermons that have information about preaching location.

In [25]:
patterns1 = ["preach't (.*?)$","preached at (.*?)$","preached in (.*?)$","preacht (.*?)$","before the (.*?)$", "before his (.*?)$"]
patterns2 = ["preacher at (.*?)$","preacher in (.*?)$","pastor at (.*?)$","pastor of (.*?)$"]
places = []
pastorPlaces = []
import csv

outFile = open(f'sermons_locations.csv','a+')
columns = ['id','title','author','date','location']
writer = csv.DictWriter(outFile, fieldnames=columns)
writer.writeheader()

located = []
for idx, title in enumerate(csv_data['title']):
    foundPattern1 = False
    for pattern in patterns1: 
        place = re.search(rf'{pattern}', title)
        if place: 
            # places.append(place.group())
            dict = {'id': tcpIDs[idx], 
                    'title': title,
                    'author': csv_data['author'][idx],
                    'date': csv_data['date'][idx],
                    'location': place.group()
                    }
            located.append(tcpIDs[idx])
            writer.writerow(dict)
            # print(f'{tcpIDs[idx]}: {place.group()}')
            foundPattern1 = True
            break
    if not foundPattern1: 
        for pattern in patterns2: 
            place = re.search(rf'{pattern}', title)
            if place: 
                dict = {'id': tcpIDs[idx], 
                    'title': title,
                    'author': csv_data['author'][idx],
                    'date': csv_data['date'][idx],
                    'location': place.group()
                }
                # pastorPlaces.append(place.group())
                located.append(tcpIDs[idx])
                writer.writerow(dict)
                foundPattern2 = True
                break

for idx, tcpID in enumerate(tcpIDs):
    if tcpID in located: 
        continue
    dict = {'id': tcpID, 
            'title': csv_data['title'][idx],
            'author': csv_data['author'][idx],
            'date': csv_data['date'][idx],
            'location': ''
            }
    writer.writerow(dict)

# print(len(places), len(pastorPlaces), len(places)+len(pastorPlaces))
# print(places)
# print(pastorPlaces)

Extracting title pages directly from TCP. 
Title pages often contain information about where the sermon was preached. 

In [90]:
import csv 
folder = f'/Users/amycweng/Digital Humanities/'
outfile = open(f'{folder}/sermon_titles.csv','a+')
columns = ['id','title page']
writer = csv.DictWriter(outfile, fieldnames=columns)
writer.writeheader()

tcpIDs = ['A65610','A15015']
for tcpID in tcpIDs: 
    path = findTextTCP(tcpID)
    with open(path,'r') as file: 
        data = file.read()
    tag = SoupStrainer("div1", attrs={'type':'title page'})
    soup = BeautifulSoup(data,parse_only=tag,features='html.parser')
    title = []
    sentences = soup.find_all('p')
    for sentence in sentences: 
        title.append(sentence.text)
    if len(title) != 0: 
        title = ' '.join(title)
        title = re.sub('\n',' ',title)
        t = re.search(r'^(.*?) \d{4}',title)
        if t: 
            writer.writerow({'id': tcpID,  'title page': t.group()})
        else: 
            writer.writerow({'id': tcpID,  'title page': title})
outfile.close()